In [1]:
import os, platform, time, urllib.request, openpyxl, operator
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
import sys, requests, re, json
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#Create Empty List
ranking = []
username = []
category = []
category_2 = []

def scrape_username(url):
    
    #accesing and parsing the input url
    response = requests.get(url)
    print(f'response {response}')
    soup = BeautifulSoup(response.content, 'html.parser')
    list_username = soup.find_all('tr')
    
    #looping to the element that we want to scrape
    for p in list(list_username):
        try:
            #getting the information (rank, names, and category)
            rank = p.find('td', 'align-middle').get_text().strip()
            ranking.append(rank)

            # i am having trouble with this part, because i could get the categories, but not the @s from the page  ):
            # tried to do the "same logic" as the code does for category, but at the end it says I have a ValueError: All arrays must be of the same length

            # name = p.find_all('a', "_blank").get_text().strip()
            # name_2 = []
            # for item in name:
            #     new_var = item.find('a', 'link').get_text()
            #     name_2.append(new_var)
            # username.append(name_2)

            name = p.find('a').get_text().strip()
            username.append(name)


            cat = p.find_all('span', 'badge badge-pill badge-light samll font-weight-normal text-muted') #i had to change this part here to get categories
            category_2 = []
            for c in cat:
                d = c.find('a', 'link').get_text()
                category_2.append(d)
            category.append(category_2)
        except:
            continue

    return ranking, username, category

#function to scrape general information
def scrape_general_info(url):
    # instagram URL 
    URL = url

    # creating a dictionary 
    data = {} 

    # getting the request from url 
    r = requests.get(URL)

    # converting the text 
    s = BeautifulSoup(r.text, "html.parser") 

    # finding meta info 
    meta = s.find("meta", property ="og:description")

    #searching followers, followeing and number of posts info
    meta_2 = meta.attrs['content']
    meta_3 = meta_2.split("-")[0].split(" ")

    # assigning the values 
    data['Followers'] = meta_3[0] 
    data['Following'] = meta_3[2] 
    data['Posts'] = meta_3[4]
    
    return data

#Create Empty List
link = []
names = []

def get_influencer_link(username):
    #to influencer url
    url = f'https://www.instagram.com/{username}/'
    driver = webdriver.Chrome()
    driver.get(url)

    time.sleep(5)

    i = 0
    while i < 8:   
        try:
            #get the links
            pages = driver.find_elements_by_tag_name('a')
            for data in pages:
                data_2 = data.get_attribute("href")
                if '/p/' in data_2:
                    link.append(data.get_attribute("href"))
                    names.append(username)
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(1)
            i += 1
        except:
            i += 1
            continue
    driver.quit()

    return link, names


likes = []
comment_counts = []
dates = []
captions = []
type_posts = []
links = []
i = 0
n = 0

def get_information(link):    
    try:
        global i, n
        
        #accessing and parsing the website url
        url = link
        response = requests.get(url)
        soup = BeautifulSoup(response.content)
        
        #find element that contain information
        body = soup.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data=json.loads(raw)
        posts =json_data['entry_data']['PostPage'][0]['graphql']
        posts= json.dumps(posts)
        posts = json.loads(posts)
        
        #acquiring information
        like = posts['shortcode_media']['edge_media_preview_like']['count']
        comment_count = posts['shortcode_media']['edge_media_to_parent_comment']['count']
        date = posts['shortcode_media']['taken_at_timestamp']
        caption = posts['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text']
        type_post = posts['shortcode_media']['__typename']
        likes.append(like)
        comment_counts.append(comment_count)
        dates.append(date)
        captions.append(caption)
        type_posts.append(type_post)
        links.append(link)
        i += 1
    except:
        i += 1
        n += 1
        print(f'number of link error {n} at iteration {i}')
        pass
    return likes, comment_counts, dates, captions, type_posts, links

In [47]:
import os, platform, time, urllib.request, openpyxl, operator
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
import sys, requests, re, json
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


In [ ]:
#scrape username instagram that we want to analyze
for i in range(10): #bcs the number of webpage is until 10
    a = i+1
    #if response 200 is that webpage can be scraped
    url = f'https://starngage.com/app/global/influencer/ranking/brazil'
    
    #call function
    ranking, username, category = scrape_username(url)

In [4]:
response = requests.get('https://starngage.com/app/global/influencer/ranking/brazil')
print(f'response {response}')
soup = BeautifulSoup(response.content, 'html.parser')
list_username = soup.find_all('tr')


response <Response [200]>


In [27]:
import re
newlist_usernames = [a.get_text(strip=True) for a in list_username]
newlist_usernames
#print(re.findall(['['@']([a-z0-9])+'], newlist_usernames))

['#@UsernameCountry/RegionTopicsFollowersEngagement Rate',
 '1Demi Lovato@ddlovatoBrazilSingerSongwritingMusicMusician141.6M0.24%',
 '2Vin Diesel@vindieselBrazilProducersDirectorActors83M0.56%',
 '3Shakira@shakiraBrazilMusicSingerMusician/Band76.2M0.81%',
 '4Shawn Mendes@shawnmendesBrazilMusicSingerSongwritingMusician70.4M2.61%',
 '5camila@camila_cabelloBrazilFashionSingerMusicModeling65.4M1.07%',
 '6Anitta 🎤@anittaBrazilArtMusicSinger63.3M0.70%',
 '7MALUMA@malumaBrazilArtMusicSinger62.8M1.10%',
 'Sign up to view the full list',
 '8Millie Bobby Brown@milliebobbybrownBrazilCelebrityActors58.7M7.18%',
 '9Whindersson Nunes@whinderssonnunesBrazilCelebrityComedyComedian58.4M0.70%',
 '10Lady Gaga@ladygagaBrazilShoppingMusicSingerSongwritingMusician53.4M0.78%',
 '11Robert Downey Jr. Official@robertdowneyjrBrazilActors53.4M1.59%',
 '12Tata Werneck@tatawerneckBrazilActorsArtist53M0.40%',
 '13James Rodríguez@jamesrodriguez10BrazilSportsSoccer49.7M0.90%',
 '14Zlatan Ibrahimović@iamzlatanibrahimo

In [49]:
inf_dict = {
    'username':username,
    'ranking':ranking,
    'category':category
}

In [50]:
#Assign to dataframe
df = pd.DataFrame(inf_dict)

#do some preprocessing
df['username_2'] = df['username'].apply(lambda x: x[1:])
df['category'] = df['category'].astype('str')
df_2 = df.drop_duplicates().reset_index(drop=True)

In [51]:
df_2

,username,ranking,category,username_2
0,,1,"['Singer', 'Songwriting', 'Music', 'Musician']",
1,,2,"['Producers', 'Director', 'Actors']",
2,,3,"['Music', 'Singer', 'Musician/Band']",
3,,4,"['Music', 'Singer', 'Songwriting', 'Musician']",
4,,5,"['Fashion', 'Singer', 'Music', 'Modeling', '']",
...,...,...,...,...
95,,96,"['Humor', 'Celebrities', 'Animals & Pets']",
96,,97,"['Art', 'Lifestyle', 'Actors', 'Celebrities']",
97,,98,['Singer'],
98,,99,"['Music', 'Singer']",


In [ ]:
# could just get information about the category using the code, and changing some details
# if you want to compare to the original one, it's on the file "test_indonesia"
# on the first cell i left some comments and also a part of code that i tried but it didn't worked. 
